In [1]:
import warnings
warnings.filterwarnings('ignore')

import os, sys
sys.path.insert(1, '../..')

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# "" = CPU
# "0" = GPU_0
# "1" = GPU_1
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 
# print( 'CUDA Version =', os.environ['CUDA_VERSION'] )

import sys
print( 'Python Version =', sys.version.split()[0] )

import tensorflow as tf
print( 'Tensorflow Version =',tf.__version__)

print("GPU Available: ", tf.test.is_gpu_available())

Python Version = 3.7.6
Tensorflow Version = 1.15.0
GPU Available:  True


In [2]:
import os

from os.path import join
from utils_1 import *
import shutil
import gc
from inspect import getsource
import time

# import wandb
# from wandb.tensorflow import WandbHook
# from wandb.keras import WandbCallback

# wandb.init(project="woundsee", sync_tensorboard=True)

In [3]:
def load_x( paths ):
    def get_x_rgb( path ):
        rgb = read_image( path, 'rgb' )
        return rgb / 255

    get_x = get_x_rgb
    x = np.array( [ get_x(path) for path in paths ] ).astype( np.float32 )
    return x

def load_y( paths ):
    def get_y( path ):
        grayscale = read_image( path, 'gray' ) / 63
        layers = [  (grayscale==i) + [0] for i in range(0, 5) ]
        
        return np.stack( layers, axis=2 )

    y = np.array( [ get_y(path) for path in paths ] ).astype( np.float32 )
    return y

In [4]:
def get_generator( this_fold_dir, t, color ):
    def my_generator( samples, batch_size, color ):
        L = len( samples )
        while True:
            batch_start = 0
            batch_end = batch_size
            shuffle(samples)
            f_paths = [ os.path.join( feature_dir, i ) for i in samples ]
            l_paths = [ os.path.join( label_dir, i ) for i in samples ]
            while batch_start < L:
                limit = min( batch_end, L )
                x = load_x( f_paths[batch_start:limit] )
                y = load_y( l_paths[batch_start:limit] )
                yield x, y
                batch_start += batch_size
                batch_end += batch_size
    
#     if not (t == "Train" or t == "Validate"):
#         raise ValueError('t must be Train or Validate')
    
    my_dir = os.path.join( this_fold_dir, t )
    feature_dir = os.path.join( my_dir, 'Features' )
    label_dir = os.path.join( my_dir, 'Labels' )
    
    
    samples = [ p for p in os.listdir( feature_dir ) if 'checkpoint' not in p]
    
    if isTest:
        samples = samples[:256]
    
    genarator = my_generator( samples, batch_size, color )
    return genarator, len(samples)

In [5]:
def train( fold ):
    
    this_ex_dir = os.path.join( this_experiment_dir, str(fold) )
    
    if not isTest:
        os.mkdir( this_ex_dir )
    
    this_fold_dir = os.path.join( dataset, str(fold) )
    
    print( "Start Process fold: %d" % (fold) )
    train_gen, num_train_samples = get_generator( this_fold_dir, "Train", 'rgb' )
    val_gen, num_val_samples = get_generator( this_fold_dir, "Validate", 'rgb' )
#     val_gen, num_val_samples = get_generator( '/notebooks/VOLUME_1TB/Thesis_dataset', "Test_Sub-Images", 'rgb' )
    
    print( "Train: %d, Val: %d" % ( num_train_samples, num_val_samples ) )

    
    callbacks = get_call_backs(this_ex_dir)
#     model     = get_model( input_height, input_width, input_chanel )
    model     = get_model( model_path )
    
    if isTest:
        model.summary()
    
    train_history = model.fit_generator( 
        generator = train_gen
        ,steps_per_epoch = np.ceil( num_train_samples / batch_size )
        ,validation_data = val_gen
        ,validation_steps = np.ceil( num_val_samples / batch_size )
        ,epochs=epochs
        ,callbacks=callbacks
    )
    
    if not isTest:
        np.save( os.path.join( this_ex_dir, 'history') , np.array(train_history.history))
    del model

In [6]:
def get_tensorboard_callback( this_ex_dir ):
    from tensorflow.keras.callbacks import TensorBoard
    path = os.path.join( os.path.join(this_ex_dir, 'tensorboard') )
    os.mkdir(path)
    print( path )
    tbCallBack = TensorBoard(log_dir=path,profile_batch=0, histogram_freq=0, write_graph=True, write_images=True)
    return tbCallBack

In [7]:
def get_adaptive_lr_callback():
    from tensorflow.keras.callbacks import LearningRateScheduler    
    adaptive_lr_callback=LearningRateScheduler(lr_scheduler)
    return adaptive_lr_callback

In [8]:
def get_call_backs(this_ex_dir):
    
    adaptive_lr = get_adaptive_lr_callback()
    callbacks = [ adaptive_lr ]
    
    if not isTest:
        log_cb = get_log_callback(this_ex_dir)
        callbacks.append( log_cb )
        
        tensorboard = get_tensorboard_callback( this_ex_dir )
        callbacks.append( tensorboard )
        
#         callbacks.append(WandbCallback())
        
    return callbacks

In [9]:
def setup_experiment():
    def create_experiment_dir():
        os.mkdir( this_experiment_dir )
        print(this_experiment_dir)
        
    def create_detail_of_experiment():
        detail_path = os.path.join( this_experiment_dir, 'detail.txt' )
        with open( detail_path, 'w', encoding="utf-8" ) as file:
            pass
        return detail_path
    
    def write_line_to_detail( msg ):
        with open( detail_path, 'a', encoding="utf-8" ) as file:
            file.write( "%s\n\n" % msg )
            
    def write_model_to_detail( model ):
        with open( detail_path, 'a', encoding="utf-8" ) as file:
            model.summary(print_fn=lambda x: file.write(x + '\n'))
            
    model = get_model(model_path)
    
    create_experiment_dir()
    detail_path = create_detail_of_experiment()
    
    write_line_to_detail( 'Method: %s' % method )
    write_line_to_detail( 'Structure: %s' % structure )
    write_line_to_detail( 'Input Shape: %s' % str( ( input_height, input_width, input_chanel) ) )
    write_line_to_detail( 'Color: %s' % color )
    write_line_to_detail( 'Description: %s' % desc )
    write_line_to_detail( 'Batch size: %d' % batch_size )
    write_line_to_detail( 'Epochs: %d' % epochs )
    write_line_to_detail( 'Start lr: %s' % learning_rate)
    
    adaptive = getsource( lr_scheduler )
    write_line_to_detail( adaptive )
    
    loss_func = getsource( loss )
    write_line_to_detail( loss_func )
    
    write_model_to_detail( model )

In [10]:
# def get_model( input_height, input_width, input_chanel ):
    
#     _dict = {
#         'u-net-res-2': get_model_unet_res_2,
#     }
    
#     if structure not in _dict.keys():
#         raise ValueError('%s not in our model.' % structure)
    
#     return _dict[structure]( input_height, input_width, input_chanel )


def get_model(model_path):
    from tensorflow.keras.models import load_model
    return load_model( model_path ,custom_objects={ 'loss_func':loss, 'loss': loss, 'dice':dice , 'iou': iou, 'cate': cate})

In [11]:
def start( start, end  ):
    
    def start_train():
        for i in range(start,end):
            print("_" * 100)
            gc.collect()
            train(i)
            print("_" * 100)
        gc.collect()
    
    def calculate_environment_name():
        global this_experiment_dir
        number_folder       = len(os.listdir(Experiments))
        this_experiment_dir = os.path.join( Experiments, str( number_folder ).zfill(4) )
        print( "Experiments: %s" % str( number_folder ).zfill(4) )
        
    calculate_environment_name()
    
    if isTest:
        start_train()
        return
    
    setup_experiment()
    start_train()

In [12]:
def get_log_callback(this_ex_dir, log_name = 'log.txt', prefix_model=''):
    log_path = os.path.join( this_ex_dir, log_name)
    from tensorflow.keras.callbacks import Callback
    from tensorflow.keras import backend as K
    
    class My_Callback_1(Callback):
        def __init__(self, filename, separator=' ; '):
            self.sep = separator
            self.filename = filename
            self.val_a = -1
            self.best_epoch = -1
            super(My_Callback_1, self).__init__()
        
        def on_epoch_begin(self, epoch, logs=None):
            self.tic = time.time()
        
        def on_epoch_end(self, epoch, logs=None):
            toc = time.time()
            t_sec = round( toc - self.tic)
            
            lr_var = self.model.optimizer.lr
            lr = K.get_value(lr_var)
            
            logs = logs or {}
            val = [ "%4d" % epoch, "%7d" % t_sec, "%10s" % lr ]
            for key in sorted( logs.keys() ):
                if key == 'lr':
                    continue
                value = logs[key]
                val.append( "%3.6f" % value )
            line = self.sep.join( val )
            with open( self.filename, 'a' ) as f:
                f.write( "\n%s" % line )
            
            if epoch+1 >= 50 and (epoch+1) % 10 == 0:
                model_name = "%s%d_model.h5" % (prefix_model,epoch+1)
                model_path = os.path.join( this_ex_dir, model_name )
                self.model.save( model_path )
                
                
            val_n = logs[ "val_dice" ]
            if val_n >= self.val_a:
                self.val_a = val_n
                self.best_epoch = epoch
                model_name = "%sbest_model.h5" % (prefix_model)
                model_path = os.path.join( this_ex_dir, model_name )
                self.model.save( model_path )
                with open( self.filename, 'a' ) as f:
                    f.write( " ***" )
                
                
        def on_train_begin(self, logs=None):
            line = "epochs" + self.sep + "secs" + self.sep + "lr" + self.sep + self.sep.join( sorted(self.params['metrics']) )
            with open( self.filename, 'w' ) as f:
                f.write( "%s" % line )
        
        def on_train_end(self, logs=None):
            model_name = "%send_model.h5" % prefix_model
            model_path = os.path.join( this_ex_dir, model_name )
            self.model.save( model_path )
            with open( self.filename, 'a' ) as f:
                f.write( "\n\nBest Epoch: %d, val__dice: %.6f" % (self.best_epoch,self.val_a) )
    
    my_callback = My_Callback_1(log_path)
    
    return my_callback

In [13]:
# def get_model_unet_res_2( input_height, input_width, input_chanel ):
#     from tensorflow.keras.layers import ZeroPadding2D, Input, Conv2D, MaxPooling2D, SpatialDropout2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Add
#     from tensorflow.keras.models import Model
#     from tensorflow.keras.optimizers import Adam
#     from tensorflow.keras.activations import relu, softmax
    
#     def residual_block( filters, kernel_size ):
#         def block( X ):
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X_shortcut = X
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = BatchNormalization() ( X )
#             X = Activation( relu ) (X)
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = Add()( [ X_shortcut, X ] )
#             X = BatchNormalization() ( X )
#             X = Activation( relu ) (X)
#             X_shortcut = X
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = BatchNormalization() ( X )
#             X = Activation( relu ) (X)
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = Add()( [ X_shortcut, X ] )
#             X = BatchNormalization() ( X )
#             X = Activation( relu ) (X)
#             return X
#         return block
    
#     def residual_block_2( filters, kernel_size ):
#         def block( X ):
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X_shortcut = X
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = Activation( relu ) (X)
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = Add()( [ X_shortcut, X ] )
#             X = Activation( relu ) (X)
#             X_shortcut = X
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = Activation( relu ) (X)
#             X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
#             X = Add()( [ X_shortcut, X ] )
#             X = Activation( relu ) (X)
#             return X
            
#         return block
    
#     # INPUT
#     input_shape = ( input_height, input_width, input_chanel )
#     input_      = Input( input_shape )
    
#     X = Conv2D( filters=16, kernel_size=(7,7), padding='same' ) ( input_ )
    
#     encode_1 = residual_block( filters=32, kernel_size=(3, 3) ) ( X )
#     pool_1 = MaxPooling2D( pool_size=(2,2) ) ( encode_1 )
    
#     encode_2 = residual_block( filters=64, kernel_size=(3, 3) ) ( pool_1 )
#     pool_2 = MaxPooling2D( pool_size=(2,2) ) ( encode_2 )
    
#     encode_3 = residual_block( filters=128, kernel_size=(3, 3) ) ( pool_2 )
#     pool_3 = MaxPooling2D( pool_size=(2,2) ) ( encode_3 )
    
#     mid_1 = residual_block( filters=256, kernel_size=(3, 3) ) (pool_3)
    
#     decode_3 = Conv2DTranspose( filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same') (mid_1)
#     decode_3 = concatenate([ encode_3, decode_3 ], axis=-1)
#     decode_3 = residual_block_2( filters=128, kernel_size=(3, 3) ) ( decode_3 )
#     decode_3 = SpatialDropout2D( 0.1 ) (decode_3)
    
#     decode_2 = Conv2DTranspose( filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same') (decode_3)
#     decode_2 = concatenate([ encode_2, decode_2 ], axis=-1)
#     decode_2 = residual_block_2( filters=64, kernel_size=(3, 3) ) ( decode_2 )
#     decode_2 = SpatialDropout2D( 0.1 ) (decode_2)
    
#     decode_1 = Conv2DTranspose( filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same') (decode_2)
#     decode_1 = concatenate([ encode_1, decode_1 ], axis=-1)
#     decode_1 = residual_block_2( filters=32, kernel_size=(3, 3) ) ( decode_1 )
    
#     output = SpatialDropout2D( 0.2 ) ( decode_1 )
#     output = Conv2D(filters=5, kernel_size=(1, 1), padding='same' ) ( output )
#     output = Activation( softmax ) (output)
    
#     model = Model( inputs=[input_], outputs=[output], name='U-Net with ResNet Block' )
#     optimizer = Adam( lr=learning_rate )
#     model.compile( optimizer=optimizer, loss=loss, metrics=[dice, iou, cate] )
    
#     return model

In [14]:
def iou(y_true, y_pred):
    from tensorflow.keras import backend as K
    smooth = 1
    y_pred = y_pred[...,1:]
    y_true = y_true[...,1:]
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    return ( intersection + smooth ) / ( union + smooth )
    

def dice(y_true, y_pred ):
    from tensorflow.keras import backend as K
    smooth = 1
    y_pred = y_pred[...,1:]
    y_true = y_true[...,1:]
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    denom = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return (2. * intersection + smooth) / (denom + smooth)
    

def cate(y_true, y_pred ):
    from tensorflow.keras.metrics import categorical_accuracy
    return categorical_accuracy(y_true, y_pred)

In [15]:
def loss(y_true, y_pred):
    from tensorflow import keras
    from tensorflow.keras import backend as K
    
    def dice(y_true, y_pred):
        y_pred = y_pred[...,1:]
        y_true = y_true[...,1:]
        intersection = K.sum(y_true * y_pred, axis=[1,2,3])
        denom = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
        d = (2. * intersection + smooth) / (denom + smooth)
        return d
    
    def iou(y_true, y_pred):
        intersection = K.sum(y_true * y_pred, axis=[1,2,3])
        union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
        return ( intersection + smooth ) / ( union + smooth )
    
    smooth = 1
    d = K.pow(1-dice(y_true, y_pred), 2)
    return d

In [16]:
def lr_scheduler(epoch, lr):
        decay_rate = .5
        decay_step = 25
        if epoch % decay_step == 0 and epoch >= 50:
            return lr * decay_rate
        return lr

In [17]:
method        = 'Sub-images'
isTest        = False
epochs        = 140
batch_size    = 6
learning_rate = 1e-04
color         = 'rgb'
structure     = 'u-net-res-2'

height = 512
width  = 512
# stride = 128


input_height, input_width, input_chanel = ( height, width, 3 )
1
dataset_name        = "wound_rajavithi_korean_medetec"
k_fold              = 10

root                = join("../../..", "data", dataset_name, "wound_segmentation", "wound_tissue")

model_path          = os.path.join("best_model_rajavithi_korean.h5")



source_dir          = join(root, "training_k_fold_with_rotation_color")

Experiments         = join(root, "experiments")

if not is_exists( Experiments ):
    os.mkdir( Experiments )

In [ ]:
if not is_exists(model_path):
    print("can't found model:", model_path)
    
else :
    for p in [ source_dir ]:
        global dataset, desc, this_experiment_dir 
        desc    = p
        dataset = p

        print(p)
        start( 2, 4+1)

../../../data/wound_rajavithi_korean_medetec/wound_segmentation/wound_tissue/training_k_fold_with_rotation_color
Experiments: 0004
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
../../../data/wound_rajavithi_korean_medetec/wound_segmentation/wound_tissue/experiments/0004
____________________________________________________________________________________________________
Start Process fold: 2
Train: 14767, Val: 1641
../../../data/wound_rajavithi_korean_medetec/wound_segmentation/wound_tissue/experiments/0004/2/ten